In [0]:
 
%sql
-- Creating a base table for the analysis of airport congestion
CREATE OR REPLACE VIEW airport_base AS
SELECT
    ORIGIN AS airport,
    TAXI_OUT,
    TAXI_IN,
    DEP_DELAY,
    CRS_DEP_TIME,
    -- Extract hour from scheduled departure time (e.g. 1330 → 13)
    HOUR(CRS_DEP_TIME) AS dep_hour
FROM flight_with_weather_2018
WHERE
    TAXI_OUT IS NOT NULL
    AND TAXI_IN IS NOT NULL
    AND CRS_DEP_TIME IS NOT NULL;


In [0]:
%sql
--4
create or replace table Airport_Congestion as 
SELECT
    airport,

    COUNT(*) AS total_flights,

    ROUND(AVG(TAXI_OUT), 2) AS avg_taxi_out,

    ROUND(AVG(TAXI_IN), 2) AS avg_taxi_in,

    ROUND(
        100.0 * SUM(CASE WHEN DEP_DELAY > 0 THEN 1 ELSE 0 END) / COUNT(*),
        2
    ) AS delay_percentage

FROM airport_base
GROUP BY airport
ORDER BY delay_percentage DESC;


num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT
    MIN(avg_taxi_out),
    MAX(avg_taxi_out),
    MIN(delay_percentage),
    MAX(delay_percentage)
FROM (
    SELECT
        airport,
        AVG(TAXI_OUT) AS avg_taxi_out,
        100.0 * SUM(CASE WHEN DEP_DELAY > 0 THEN 1 ELSE 0 END) / COUNT(*) AS delay_percentage
    FROM airport_base
    GROUP BY airport
);

MIN(avg_taxi_out),MAX(avg_taxi_out),MIN(delay_percentage),MAX(delay_percentage)
6.8882352941176475,27.482461161729397,9.07821229050279,60.05560595172178


In [0]:
%sql
-- 3Time-of-Day Congestion Analysis
create or replace table Time_of_Day_Congestion_Analysis as
SELECT
    airport,
    dep_hour,

    COUNT(*) AS total_flights,

    ROUND(AVG(TAXI_OUT), 2) AS avg_taxi_out,

    ROUND(AVG(TAXI_IN), 2) AS avg_taxi_in,

    ROUND(
        100.0 * SUM(CASE WHEN DEP_DELAY > 0 THEN 1 ELSE 0 END) / COUNT(*),
        2
    ) AS delay_percentage

FROM airport_base
GROUP BY airport, dep_hour
ORDER BY airport, dep_hour;


num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Peak Congestion Hour Per Airport
SELECT *
FROM (
    SELECT
        airport,
        dep_hour,
        AVG(TAXI_OUT) AS avg_taxi_out,

        RANK() OVER (
            PARTITION BY airport
            ORDER BY AVG(TAXI_OUT) DESC
        ) AS congestion_rank
    FROM airport_base
    GROUP BY airport, dep_hour
)



airport,dep_hour,avg_taxi_out,congestion_rank
ABE,19,22.0,1
ABE,21,20.0,2
ABE,5,19.257918552036198,3
ABE,17,17.172514619883042,4
ABE,13,17.161943319838056,5
ABE,6,17.105568445475637,6
ABE,18,16.246031746031747,7
ABE,12,16.20820189274448,8
ABE,7,15.728813559322035,9
ABE,8,14.272727272727273,10


In [0]:
%sql
-- Identify Peak Congestion Hour Per Airport (Global Ranking)
SELECT
    airport,
    dep_hour AS peak_dep_hour,
    avg_taxi_out,
    RANK() OVER (ORDER BY avg_taxi_out DESC) AS airport_congestion_rank
FROM (
    SELECT
        airport,
        dep_hour,
        AVG(TAXI_OUT) AS avg_taxi_out,
        dense_rank() OVER (
            PARTITION BY airport
            ORDER BY AVG(TAXI_OUT) DESC, dep_hour
        ) AS rn
    FROM airport_base
    GROUP BY airport, dep_hour
)
WHERE rn = 1
ORDER BY airport_congestion_rank;


airport,peak_dep_hour,avg_taxi_out,airport_congestion_rank
GRI,10,44.0,1
IDA,6,42.57142857142857,2
MMH,11,40.0,3
ISN,7,38.276923076923076,4
STX,11,38.0,5
DLH,20,37.0,6
FAT,1,35.0,7
ITH,11,34.0,8
IMT,17,34.0,8
JFK,19,33.91507252470044,10


In [0]:
%sql
-- ============================================================
-- 2GLOBAL PEAK HOUR CONGESTION TABLE
-- Identifies system-wide congestion patterns by hour
-- ============================================================

CREATE OR REPLACE TABLE global_peak_hour_congestion AS
SELECT
    dep_hour,

    -- Total number of flights in this hour
    COUNT(*) AS total_flights,

    -- Average taxi-out time (proxy for ground congestion)
    ROUND(AVG(TAXI_OUT), 2) AS avg_taxi_out,

    -- Average departure delay in minutes
    ROUND(AVG(DEP_DELAY), 2) AS avg_dep_delay

FROM airport_base
GROUP BY dep_hour;


num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT *
FROM global_peak_hour_congestion
ORDER BY avg_taxi_out DESC;


dep_hour,total_flights,avg_taxi_out,avg_dep_delay
8,468967,19.0,4.44
9,410230,18.47,5.41
18,396444,18.19,17.02
17,451931,17.88,15.29
20,335947,17.8,16.08
10,424309,17.71,6.56
15,420645,17.67,12.7
7,473673,17.63,3.1
11,439489,17.43,7.88
16,402904,17.34,13.8


# Efficiency Score Calculation #

In [0]:
%sql
-- Airport-level aggregation (base metrics)
SELECT
    airport,
    AVG(TAXI_OUT) AS avg_taxi_out,
    AVG(TAXI_IN) AS avg_taxi_in,
    SUM(CASE WHEN DEP_DELAY > 0 THEN 1 ELSE 0 END) * 1.0 / COUNT(*) AS delay_ratio
FROM airport_base
GROUP BY airport;

airport,avg_taxi_out,avg_taxi_in,delay_ratio
MSO,15.59178669935642,8.17958933496782,0.2482378179589335
MDW,12.367941755121755,5.450113685897058,0.6005560595172178
PDX,13.621160190909382,7.655690444921362,0.2909286011723630
CLT,20.213488315993086,6.283243914533198,0.3868342170978276
TVC,16.376405963902695,10.810096782631442,0.2660214491237248
DCA,19.419018830293634,7.654853260827224,0.3232235094307284
FAT,15.621992191046944,9.549441569054753,0.2635975665123036
ONT,11.14620382226016,6.667804808649879,0.3725518091715275
PBG,15.152487961476725,10.87158908507223,0.2616372391653291
ESC,15.346972176759412,8.713584288052374,0.2536824877250409


In [0]:
%sql
-- Normalize the metrics

WITH airport_metrics AS (
    SELECT
        airport,
        AVG(TAXI_OUT) AS avg_taxi_out,
        AVG(TAXI_IN) AS avg_taxi_in,
        SUM(CASE WHEN DEP_DELAY > 0 THEN 1 ELSE 0 END) * 1.0 / COUNT(*) AS delay_ratio
    FROM airport_base
    GROUP BY airport
)
SELECT
    airport,
    avg_taxi_out,
    avg_taxi_in,
    delay_ratio,

    avg_taxi_out / MAX(avg_taxi_out) OVER () AS taxi_out_norm,
    avg_taxi_in / MAX(avg_taxi_in) OVER () AS taxi_in_norm,
    delay_ratio / MAX(delay_ratio) OVER () AS delay_norm

FROM airport_metrics;


airport,avg_taxi_out,avg_taxi_in,delay_ratio,taxi_out_norm,taxi_in_norm,delay_norm
MSO,15.59178669935642,8.17958933496782,0.2482378179589335,0.5673358949768555,0.49621917635633134,0.413347
MDW,12.367941755121755,5.450113685897058,0.6005560595172178,0.4500303550813232,0.33063407140779727,1.000000
PDX,13.621160190909382,7.655690444921362,0.2909286011723630,0.4956310175697612,0.46443656905580566,0.484432
CLT,20.213488315993086,6.283243914533198,0.3868342170978276,0.7355050261707021,0.3811763638043157,0.644127
TVC,16.376405963902695,10.810096782631442,0.2660214491237248,0.5958857129836538,0.6558003222579517,0.442959
DCA,19.419018830293634,7.654853260827224,0.3232235094307284,0.7065967897131251,0.4643857808334831,0.538207
FAT,15.621992191046944,9.549441569054753,0.2635975665123036,0.5684349774612361,0.5793219972305498,0.438922
ONT,11.14620382226016,6.667804808649879,0.3725518091715275,0.4055751687109365,0.40450595680987755,0.620345
PBG,15.152487961476725,10.87158908507223,0.2616372391653291,0.551351200764263,0.6595307857836664,0.435658
ESC,15.346972176759412,8.713584288052374,0.2536824877250409,0.5584278673749491,0.5286142667388362,0.422413


In [0]:
%sql
-- 1Final Congestion Score + Ranking (THIS IS THE OUTPUT)
-- Weight logic (simple & defensible):
-- Taxi-Out → 40%
-- Taxi-In → 30%
-- Delay Ratio → 30%
-- ============================================================
-- FINAL AIRPORT CONGESTION SCORE TABLE (STORED)
-- ============================================================

CREATE OR REPLACE TABLE airport_congestion_final AS
SELECT
    airport,

    -- Final congestion score
    ROUND(
        (0.4 * taxi_out_norm) +
        (0.3 * taxi_in_norm) +
        (0.3 * delay_norm),
        3
    ) AS congestion_score,

    -- Congestion ranking (1 = most congested)
    RANK() OVER (
        ORDER BY
            (0.4 * taxi_out_norm) +
            (0.3 * taxi_in_norm) +
            (0.3 * delay_norm) DESC
    ) AS congestion_rank

FROM (
    SELECT
        airport,

        -- Normalize taxi-out time
        AVG(TAXI_OUT) / MAX(AVG(TAXI_OUT)) OVER () AS taxi_out_norm,

        -- Normalize taxi-in time
        AVG(TAXI_IN) / MAX(AVG(TAXI_IN)) OVER () AS taxi_in_norm,

        -- Normalize delay ratio
        (SUM(CASE WHEN DEP_DELAY > 0 THEN 1 ELSE 0 END) * 1.0 / COUNT(*))
            / MAX(
                SUM(CASE WHEN DEP_DELAY > 0 THEN 1 ELSE 0 END) * 1.0 / COUNT(*)
            ) OVER () AS delay_norm

    FROM airport_base
    GROUP BY airport
);



num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT *
FROM airport_congestion_final
limit 10;

airport,congestion_score,congestion_rank
MMH,0.729,1
JFK,0.711,2
LGA,0.694,3
SUX,0.681,4
CMX,0.674,5
ELM,0.671,6
EWR,0.657,7
TTN,0.657,8
DBQ,0.652,9
EAU,0.65,10


In [0]:
%sql
-- Efficiency Score = 1 - Congestion Score
SELECT
    airport,

    ROUND(
        1 - (
            (0.4 * taxi_out_norm) +
            (0.3 * taxi_in_norm) +
            (0.3 * delay_norm)
        ),
        3
    ) AS efficiency_score,

    RANK() OVER (
        ORDER BY
            1 - (
                (0.4 * taxi_out_norm) +
                (0.3 * taxi_in_norm) +
                (0.3 * delay_norm)
            ) DESC
    ) AS efficiency_rank

FROM (
    SELECT
        airport,

        AVG(TAXI_OUT) / MAX(AVG(TAXI_OUT)) OVER () AS taxi_out_norm,
        AVG(TAXI_IN) / MAX(AVG(TAXI_IN)) OVER () AS taxi_in_norm,
        (SUM(CASE WHEN DEP_DELAY > 0 THEN 1 ELSE 0 END) * 1.0 / COUNT(*))
            / MAX(SUM(CASE WHEN DEP_DELAY > 0 THEN 1 ELSE 0 END) * 1.0 / COUNT(*)) OVER ()
            AS delay_norm

    FROM airport_base
    GROUP BY airport
)
ORDER BY efficiency_rank;


airport,efficiency_score,efficiency_rank
SCC,0.751,1
BRW,0.747,2
WRG,0.719,3
PSG,0.714,4
YAK,0.703,5
ADQ,0.693,6
OME,0.693,7
BET,0.692,8
LWS,0.691,9
OTZ,0.684,10


# FINAL GOLD QUERY #

In [0]:
%sql
-- ============================================================
-- STORE FINAL AIRPORT EFFICIENCY RESULT AS A TABLE
-- ============================================================

CREATE OR REPLACE TABLE airport_efficiency_powerbi AS

WITH peak_hour AS (
    SELECT
        airport,
        dep_hour AS peak_dep_hour,
        avg_taxi_out AS peak_avg_taxi_out
    FROM (
        SELECT
            airport,
            dep_hour,
            AVG(TAXI_OUT) AS avg_taxi_out,
            ROW_NUMBER() OVER (
                PARTITION BY airport
                ORDER BY AVG(TAXI_OUT) DESC, dep_hour
            ) AS rn
        FROM airport_base
        GROUP BY airport, dep_hour
    )
    WHERE rn = 1
),

efficiency AS (
    SELECT
        airport,
        ROUND(
            1 - (
                (0.4 * taxi_out_norm) +
                (0.3 * taxi_in_norm) +
                (0.3 * delay_norm)
            ),
            3
        ) AS efficiency_score
    FROM (
        SELECT
            airport,
            AVG(TAXI_OUT) / MAX(AVG(TAXI_OUT)) OVER () AS taxi_out_norm,
            AVG(TAXI_IN) / MAX(AVG(TAXI_IN)) OVER () AS taxi_in_norm,
            (SUM(CASE WHEN DEP_DELAY > 0 THEN 1 ELSE 0 END) * 1.0 / COUNT(*))
                / MAX(SUM(CASE WHEN DEP_DELAY > 0 THEN 1 ELSE 0 END) * 1.0 / COUNT(*)) OVER ()
                AS delay_norm
        FROM airport_base
        GROUP BY airport
    )
)

SELECT
    e.airport,
    e.efficiency_score,
    RANK() OVER (ORDER BY e.efficiency_score DESC) AS efficiency_rank,
    p.peak_dep_hour,
    ROUND(p.peak_avg_taxi_out, 2) AS peak_avg_taxi_out
FROM efficiency e
JOIN peak_hour p
    ON e.airport = p.airport;


num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT *
FROM airport_efficiency_powerbi
LIMIT 10;


airport,efficiency_score,efficiency_rank,peak_dep_hour,peak_avg_taxi_out
SCC,0.751,1,10,9.18
BRW,0.747,2,12,10.47
WRG,0.719,3,10,8.36
PSG,0.714,4,15,9.08
YAK,0.703,5,18,9.41
OME,0.693,6,9,16.88
ADQ,0.693,6,8,9.5
BET,0.692,8,8,10.21
LWS,0.691,9,12,12.35
OTZ,0.684,10,20,9.83
